## Heston Calibration + IV Surface (Moneyness vs T)

This notebook:
- charge les prix de calls depuis `options_data.csv`,
- calibre les paramètres de Heston avec PyTorch + autograd,
- utilise ces paramètres pour calculer les prix de call sur une grille (moneyness, maturité),
- convertit ces prix en IV Black-Scholes et trace une surface 3D.

In [15]:
import math
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import torch

# Chemins et import du pricer Heston torch
import sys
root = Path.cwd()  # répertoire racine du projet (IV_Euro)
sys.path.append(str(root / 'Heston' / 'NN'))

from heston_torch import HestonParams, carr_madan_call_torch

torch.set_default_dtype(torch.float64)
device = torch.device('cpu')

CSV_PATH = root / 'options_data.csv'
df = pd.read_csv(CSV_PATH)
required = {'S0', 'K', 'T', 'C_mkt'}
missing = required - set(df.columns)
if missing:
    raise ValueError(f'Missing columns in options_data.csv: {missing}')
df = df[list(required)].dropna().copy()
n_total = len(df)
max_points = 300
if n_total > max_points:
    df = df.sort_values('T')
    idx = np.linspace(0, n_total - 1, max_points, dtype=int)
    df = df.iloc[idx]
df = df.reset_index(drop=True)
print(f'Calibration on {len(df)} quotes (out of {n_total}).')
df.head()

Calibration on 300 quotes (out of 895).


,C_mkt,S0,T,K
0,93.78,671.929993,0.004065,570.0
1,85.62,671.929993,0.004065,580.0
2,58.22,671.929993,0.004065,615.0
3,37.00,671.929993,0.004065,635.0
4,29.92,671.929993,0.004065,640.0


In [16]:
# Préparation des tenseurs pour la calibration
S0_t = torch.tensor(df['S0'].to_numpy(), dtype=torch.float64, device=device)
K_t = torch.tensor(df['K'].to_numpy(), dtype=torch.float64, device=device)
T_t = torch.tensor(df['T'].to_numpy(), dtype=torch.float64, device=device)
C_mkt_t = torch.tensor(df['C_mkt'].to_numpy(), dtype=torch.float64, device=device)

r = 0.02  # taux sans risque constant
q = 0.0   # dividende nul
print('T range: ', float(T_t.min()), '→', float(T_t.max()))

T range:  0.0040652934562721 → 0.8396817318124365


In [17]:
def prices_from_unconstrained(u: torch.Tensor) -> torch.Tensor:
    """Map un vecteur u∈R⁵ vers des HestonParams valides, puis price tous les calls."""
    params = HestonParams.from_unconstrained(u[0], u[1], u[2], u[3], u[4])
    prices = []
    for S0_i, K_i, T_i in zip(S0_t, K_t, T_t):
        price_i = carr_madan_call_torch(S0_i, r, q, T_i, params, K_i)
        prices.append(price_i)
    return torch.stack(prices)

def loss(u: torch.Tensor) -> torch.Tensor:
    model_prices = prices_from_unconstrained(u)
    diff = model_prices - C_mkt_t
    return 0.5 * (diff ** 2).mean()

In [18]:
# Descente de gradient avec Adam sur u (paramètres non contraints)
u = torch.tensor([1.0, -3.0, -0.5, -0.5, -3.0], dtype=torch.float64, device=device, requires_grad=True)
optimizer = torch.optim.Adam([u], lr=5e-3)
max_iters = 60
history = []
for it in range(max_iters):
    optimizer.zero_grad()
    L = loss(u)
    L.backward()
    optimizer.step()
    history.append(float(L.detach().cpu()))
    if it % 10 == 0:
        print(f'Iter {it:03d} | loss = {history[-1]:.6e}')
history[-5:]

Iter 000 | loss = 1.617958e+02
Iter 010 | loss = 1.608774e+02
Iter 020 | loss = 1.603515e+02
Iter 030 | loss = 1.599926e+02
Iter 040 | loss = 1.596212e+02
Iter 050 | loss = 1.592376e+02


[159.05048157956446,
 159.01351121995214,
 158.97667831622314,
 158.93997662786234,
 158.90340053867354]

In [19]:
# Paramètres calibrés
with torch.no_grad():
    params_fin = HestonParams.from_unconstrained(u[0], u[1], u[2], u[3], u[4])
calib = {
    'kappa': float(params_fin.kappa.cpu()),
    'theta': float(params_fin.theta.cpu()),
    'sigma': float(params_fin.sigma.cpu()),
    'rho': float(params_fin.rho.cpu()),
    'v0'  : float(params_fin.v0.cpu()),
}
pd.Series(calib, name='Calibrated Heston parameters')

kappa    1.132673
theta    0.047996
sigma    0.582199
rho     -0.374231
v0       0.046052
Name: Calibrated Heston parameters, dtype: float64

In [20]:
# Construction de la surface IV (moneyness, T) à partir des prix Heston
from math import erf

def bs_price(S0, K, T, vol, r):
    if T <= 0 or vol <= 0:
        return max(0.0, S0 - K * math.exp(-r * T))
    sqrt_T = math.sqrt(T)
    v = vol * sqrt_T
    d1 = (math.log(S0 / K) + (r + 0.5 * vol * vol) * T) / v
    d2 = d1 - v
    nd1 = 0.5 * (1.0 + erf(d1 / math.sqrt(2.0)))
    nd2 = 0.5 * (1.0 + erf(d2 / math.sqrt(2.0)))
    return S0 * nd1 - K * math.exp(-r * T) * nd2

def implied_vol(price, S0, K, T, r, tol=1e-6, max_iter=100):
    intrinsic = max(0.0, S0 - K * math.exp(-r * T))
    if price <= intrinsic + 1e-12:
        return 0.0
    low, high = 1e-6, 1.0
    p_high = bs_price(S0, K, T, high, r)
    while p_high < price and high < 5.0:
        high *= 2.0
        p_high = bs_price(S0, K, T, high, r)
    if p_high < price:
        return float('nan')
    for _ in range(max_iter):
        mid = 0.5 * (low + high)
        p_mid = bs_price(S0, K, T, mid, r)
        if abs(p_mid - price) < tol:
            return mid
        if p_mid > price:
            high = mid
        else:
            low = mid
    return 0.5 * (low + high)

S0_ref = float(df['S0'].median())
T_grid = np.arange(0.1, 2.6, 0.1)
m_grid = np.linspace(0.8, 1.2, 21)
iv_surface = np.zeros((len(T_grid), len(m_grid)))

with torch.no_grad():
    params_tensor = params_fin
    for i, T_val in enumerate(T_grid):
        K_vec = torch.tensor(S0_ref * m_grid, dtype=torch.float64, device=device)
        prices_t = carr_madan_call_torch(torch.tensor(S0_ref, dtype=torch.float64, device=device), r, 0.0, T_val, params_tensor, K_vec)
        prices = prices_t.cpu().numpy()
        for j, price in enumerate(prices):
            iv_surface[i, j] = implied_vol(price, S0_ref, float(S0_ref * m_grid[j]), T_val, r)
iv_surface[:3, :3]

array([[0.27527119, 0.2684591 , 0.26163723],
       [0.27027644, 0.26352217, 0.2567465 ],
       [0.26549389, 0.25880771, 0.2520991 ]])

In [21]:
# Plot 3D IV surface (moneyness, T, IV)
MM, TT = np.meshgrid(m_grid, T_grid)
fig = go.Figure(data=[go.Surface(x=MM, y=TT, z=iv_surface, colorscale='Viridis')])
fig.update_layout(
    title='Heston Implied Vol Surface (from calibrated params)',
    scene=dict(
        xaxis_title='Moneyness K/S0',
        yaxis_title='Time to Maturity T (years)',
        zaxis_title='Implied Volatility',
    ),
    width=900,
    height=600,
)
fig.show()